<a href="https://colab.research.google.com/github/w95wayne10/personal_financial_management/blob/master/DH_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A hedger is short 10,000 European calls.  
As an option covers 100 shares of stock, N = 1,000,000.  


0 3 6 3

In [1]:
import numpy as np

In [21]:
# 2: 0: 0.41631 and 0.46255
path = [ 
 {0:(0.7225, 0.50397)},
 {-3:(0.1726,0.19247),0:(0.55274,0.50304),3:(1.39859,0.81218)},
 {-6:(0.0,0.0), -3:(0.0, 0.0), 0:(0.41631, 0.50304), 3:(1.22372, 1.0), 6:(2.46242, 1.0)},
 {-9:(0, 1.0), -6:(0,1.0), -3:(0, 1.0), 0:(0.0, 1.0), 3:(1.22372, 1.0), 6:(2.46242, 1.0), 9:(3.71628, 1.0)}
]

In [ ]:
price_range = np.exp(np.log(100)+(np.arange(9)-4)*3*np.sqrt(0.00006575)/2)

In [33]:
#yt1 = yt0 + r - ht**2/2 + ht*epsilon
#Monte Carlo
S = 100
y0 = np.log(S)
yt = y0
h=np.sqrt(10.96/1e5)
#np.random.seed(1)
print(np.exp(yt), h**2*1e5)
price_path = []
price_path.append(S)
for t in range(3):
  eps = np.random.standard_normal(1)[0]
  yt = yt - h**2/2 + h*eps
  h = np.sqrt(0.000006575 + 0.9*h**2 + 0.04*h**2*eps**2)
  price_path.append(np.exp(yt))
  print(np.exp(yt), h**2*1e5, eps)
node_path = []
for idx, p in enumerate(price_path):
  a = idx*2+1
  b = a//2
  node_path.append(int((np.argmin(np.abs(p - np.exp(np.log(100)+(np.arange(a)-b)*3*np.sqrt(0.00006575)/2)))-b)*3))
node_path = np.array(node_path)
node_path

100.00000000000004 10.959999999999999
99.73380902561811 10.548762160098264 -0.2493703754774101
101.23746336093711 11.05341465538064 1.462107937044974
99.06282826180312 12.48208033266524 -2.060140709497654


array([ 0,  0,  3, -3])

In [34]:
#hedge
S = 100
N = 1000000
node_index = node_path.copy()
y = np.log(S)+node_index*np.sqrt(0.00006575)/2
S = np.exp(y)
temp = []
for idx, node in enumerate(node_index):
  temp.append(path[idx][node])
f,delta = list(zip(*temp))
f = np.array(f)
delta = np.array(delta)
change_delta = delta.copy()
change_delta[1:] = delta[1:]-delta[:-1]
shares_bought = N*change_delta
cost_shares = S*shares_bought
cumulative_cost = np.cumsum(cost_shares)
cumulative_cost[-1] - N*S[0]

608141.1640226096